In [72]:
import pandas as pd
import re
import nltk
import contractions

tokenizer = nltk.RegexpTokenizer(r"\w+")
lemmatizer = nltk.WordNetLemmatizer()

SUBREDDIT = 'ukraine'
filename = f'./data/{SUBREDDIT}_submissions_redditapi.csv'
dataframe = pd.read_csv(filename)
wordlist_file = open(f"./data/ukraine_conflict_words.txt", "r")
wordlist = wordlist_file.read().split("\n")
wordlist = wordlist[:-1]

preprocessing with
* lowercase
* remove urls, usertags
* join contractions
* remove punctuation
* tokenize
* lemmatize

In [73]:
def preprocessing(submission):
    submission = str(submission)
    submission = submission.lower()
    #source https://www.regextester.com/93652
    submission = re.sub(r"(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?","",submission)
    submission = re.sub(r"(@\[A-Za-z0-9]+)|(\w+:\/\/\S+)|^&gt;","",submission)
    submission = " ".join(contractions.fix(word) for word in submission.split())

    submission = re.sub(r"[^0-9A-Za-z \t]","",submission)
    submission = tokenizer.tokenize(submission)    
    submission = " ".join([lemmatizer.lemmatize(word, pos=get_pos_tag([word])) for word in submission])
    return submission

In [74]:
def get_pos_tag(word):
    pos = nltk.pos_tag(word)
    if pos[0][1].startswith('J'):
        return 'a'
    elif pos[0][1].startswith('V'):
        return 'v'
    elif pos[0][1].startswith('R'):
        return 'r'
    else :
        return 'n'

apply preprocessing on title and selftext

In [75]:
dataframe['title'] = dataframe['title'].apply(preprocessing)
dataframe['self_text'] = dataframe['self_text'].apply(preprocessing)

filter only for submissions where either title or self text contains certain ukraine war related words

In [76]:
print(dataframe.shape)

filtered_dataframe = dataframe[dataframe["title"].str.contains('|'.join(wordlist)) | dataframe["self_text"].str.contains('|'.join(wordlist))]
print(filtered_dataframe.shape)

filtered_dataframe.to_csv(f'./data/{SUBREDDIT}_filtered.csv', header=True, index=False, columns=list(dataframe.axes[1]), encoding='utf-8')

(36738, 12)
(25567, 12)
